In [1]:
import requests
import pickle
import sys
sys.path.append('/Users/michael/Code/UkuNet/')

from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from UkuNet import scrape_artists, scrape_songs, utilities, parser

pd.options.display.max_colwidth = 1000

In [2]:
artists_dict = scrape_artists.get_artists()

100%|██████████| 26/26 [00:15<00:00,  1.69it/s]

Found 1883 artists


In [3]:
songs_dict = scrape_songs.get_songs()

100%|██████████| 26/26 [16:54<00:00, 39.00s/it]

Found 5588 songs.


In [2]:
songs_dict = utilities.load_json('songs')

In [3]:
df = pd.DataFrame(columns=['song', 'artist', 'genre', 'parts', 'chords', 'lyrics'])

In [4]:
errors = []

for artist, l in tqdm(songs_dict.items()):
    for url in l:
        r = requests.get(url)
        song = url.split('/')[-2]
        
        if r.status_code != 200:
            raise Exception("Wrong Response.")
            
        try:
            utilities.store_page(r.text, song)
            genre = parser.get_genre(r.text)

            my_parser = parser.MyHTMLParser(artist, song, genre)
            my_parser.feed(r.text)
            df = df.append(my_parser.dump(), ignore_index=True)
        except Exception as e:
            errors.append([artist, song, e])

100%|██████████| 1883/1883 [1:03:18<00:00,  2.02s/it]


In [7]:
len(errors)

107

In [9]:
assert(len(errors) + len(df) == 5588)

In [12]:
df.to_csv(utilities.ROOT_DIR + '/data/chords_dataframe.csv')

In [13]:
df.head()

,song,artist,genre,parts,chords,lyrics
0,end-of-me,a-day-to-remember,"[emocore, hardcore, pop punk, post-hardcore, screamo]","[Intro, Interlude, Interlude, Outro]","[[[Bm, Gbm, E, Bm, Gbm], [E], [Bm], [Gbm, E], [Bm, Gbm, E], [Bm], [Gbm, E], [Bm], [Gbm, E], [Bm], [Gbm, E], [Bm], [Gbm, E], [Bm, Gbm, E]], [[Bm, Gbm, E, Bm, Gbm], [E], [Bm], [Gbm, E], [Bm, Gbm], [E], [Bm], [Gbm, E], [Bm], [Gbm, E], [Bm], [Gbm, E], [Bm], [Gbm, E], [Bm, Gbm, E]], [[Bm, Gbm, A, E, Bm, Gbm], [A, E], [Bm, Gbm], [A, E], [Bm], [Gbm, E], [Bm], [Gbm, E], [Bm], [Gbm, E], [Bm, Gbm, E]], []]","[[-x2-:, I took some time away, For understanding, Of what brought us here today, Oh, you'll be the end of me, I'm never happy like we were happy, I'm never sad like we were sad, Oh, you'll be the end of me, You know me all too well, And I can't suppress the memories, You're gone and I can tell, That I've lost more than you'll ever see, Don't say that it's not fair, That you're not the person you wanna be, Oh, you'll be the end of me], [-x2-:, I know I wasn't perfect,, But I kept trying, ‘Till I forgot why I stayed, Oh, you'll be the end of me, Yeah, we lost all meaning, We lost the magic, So good luck to you on your way, Oh, you'll be the end of me, You know me all too well, And I can't suppress the memories, You're gone and I can tell, That I've lost more than you'll ever see, Don't say that it's not fair, That you're not the person you wanna be, Oh, you'll be the end of me You'll be the end of me], [-x2-:, I get it now, I get it, Gave more than I took back, oh well -so back away..."
1,the-downfall-of-us-all,a-day-to-remember,"[emocore, hardcore, pop punk, post-hardcore, screamo]","[Intro, Interlude, Interlude]","[[[C, C, Am, F], [C], [Am], [F, C], [C], [Am, F, Am], [Am, Ab], [F, C, Am], [Ab, C], [C], [Am], [F, C], [C], [Am], [F, G], [F, C]], [[C, C, Am, F, C], [Am, F], [C], [Am, G], [Am, Ab], [F, C, Am], [Ab, C], [C], [Am], [F, C], [C], [Am], [F, G], [F, C]], [[C, C, Am, F, C], [Am, F], [C], [Am, F], [C], [Am], [F, C], [C], [Am], [F, C], [C], [Am], [F, C], [C], [Am], [F, G], [F, G], [F, C], [C], [C], [C], [C]]]","[[: Da da da da dada dada Da da da dada dada Dada dada dada dada dada dada Dada dada LET'S GO!!!, -x2-, It's not easy making a name for yourself, Where do you draw the line?, I never thought I'd be in this far, Let's have some fun and never change, not for anyone, Try not to miss me when I'm gone, Whooa, Whoooa, I sold my soul to the open road, -I'll live my life alone-, You won't find me in the same spot, Believe me I could never stop, My life's turned upside down, Meet me out past the train tracks, I'm leaving and not coming back, You're right and I was wrong, This town will be the downfall of us all], [-x2-:, I'm gonna need you to try to follow along., I did the best that I could to try to write you songs., Now go tell them we sold out, like we're the ones who changed., I write what I feel, I say what I mean. You can't buy sincerity., -Whooa, Whoooa-, I sold my soul to the open road, -I'll live my life alone-, You won't find me in the same spot, Believe me I could never stop, My..."
2,another-song-about-the-weekend,a-day-to-remember,"[emocore, hardcore, pop punk, post-hardcore, screamo]","[Verse, Pre-Chorus, Chorus, Verse, Pre-Chorus, Chorus, Bridge, Chorus]","[[[C, F], [C], [G], [C, F], [C], [G]], [[Am], [C], [F], [C, G]], [[C], [F, Am], [G, F], [C], [F], [Am, G, F]], [[C, F], [C, G]], [[Am], [C], [F], [C, G], [Am, C], [F, C, G]], [[C], [F, Am], [G, F], [C], [F], [Am, G, F]], [[C], [F]], [[C], [F, Am], [G, F], [C], [F], [Am, G, F]]]","[[They keep playing sad songs on the radio, And I feel like I'm so alone, On this 15 hour drive, And all the while I tell myself to just believe, Cause nobody can give so much, And never get anything -never get anything-], [Everyone I used to know, says they don't know what i've become, But I'm still the same Not much has changed, I still know where I came from], [I fell asleep with the lights on, And I can see that you're t